<a href="https://www.kaggle.com/code/suehuynh/flood-prediction-eda-xgboost-ensemble?scriptVersionId=180634669" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Flood Prediction - Kaggle Playground May 2024

Goal: The goal of this competition is to predict the probability of a region flooding based on various factors.

# Preparation
### Import libraries

In [ ]:
import pandas as pd              # For data manipulation and analysis
import numpy as np               # For numerical computing
from datetime import datetime
import scipy.stats as stats      # For statistical analysis
import math
import matplotlib                # For plotting and visualization
import matplotlib.pyplot as plt  
from pandas.plotting import parallel_coordinates
import seaborn as sns            # For statistical data visualization
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

### Load datasets

In [ ]:
df_train = pd.read_csv('/kaggle/input/playground-series-s4e5/train.csv',index_col=0)
df_test = pd.read_csv('/kaggle/input/playground-series-s4e5/test.csv', index_col=0)
sample_submission = pd.read_csv('/kaggle/input/playground-series-s4e5/sample_submission.csv',index_col=0)

# Exploratory Data Analysis

In [ ]:
df_train.info()

In [ ]:
df_test.info()

In [ ]:
df_train.head()

In [ ]:
df_train.nunique()

**Observations:**
- Train dataset has 1,117,957 rows x 21 columns, of 20 predictors and FloodProbability being the target for the model.
- Test dataset has 745,305 rows x 21 columns of 20 predictors and 1 ID column.
- Both datasets are very structured and 'clean' with no missing values.
- All the predictors are numerical and specifically, integers.


In [ ]:
df_train.describe().T.style

# Data Visualization
### Univariate Analysis
First, let's look at the distribution of each predictor.

In [ ]:
sns.histplot(data = df_train['FloodProbability'], bins = 20)

In [ ]:
fig, axes = plt.subplots(5, 4, figsize=(20, 25))
 
for i, column in enumerate(df_train.columns):
    if column == 'FloodProbability':
        continue
    plt.subplots_adjust(top = 0.85)
    ax = sns.histplot(data = df_train, 
                x = column, 
                bins = df_train[column].nunique(),
                ax = axes[i // 4, i % 4])
    
    ax.set_yticklabels(['{:,.0f}K'.format(ticks / 1000) for ticks in ax.get_yticks()])
fig.tight_layout(h_pad = 2)
plt.subplots_adjust(top = 0.95)
plt.suptitle('Distribution of Flood Predictors', fontsize = 14)
plt.show()

**Observation**: The predictors are slightly right-skew normally distributed with the mean around 4.0 to 6.0.

### Multivariate Analysis
Next, let's examine the correlation of predictor-to-predictor and predictor-to-target using correlation heatmap.

In [ ]:
# Correlation Matrix Heatmap
fig, ax = plt.subplots(figsize = (12,10))
corr = df_train.corr()
hm = sns.heatmap(corr,
                annot = True,
                ax = ax,
                cmap = 'Blues',
                fmt = '.2f')
fig.subplots_adjust(top = 0.95)
plt.suptitle('Flood Predictors Correlation Heatmap', fontsize = 14)
plt.show()

**Observations:**
- There are very low correlation among the predictors, showing their independence in the model. We will not need to remove any of them!
- Each predictor has the similar correlation with the target at around 0.18 to 0.19.

**Actions:**
- Visualize the relationship of each predictor-to-target pairs to uncover patterns using scatterplots.

In [ ]:
fig, axes = plt.subplots(5, 4, figsize=(20, 25))

for i, column in enumerate(df_train.columns):
    if column == 'FloodProbability':
        continue
    temp_df = df_train[['FloodProbability', column]].groupby(column).mean().reset_index()
    plt.subplots_adjust(top = 0.85)
    ax = sns.scatterplot(data = temp_df,
                y = column,
                x = 'FloodProbability',
                ax = axes[i // 4, i % 4])

fig.tight_layout(h_pad = 2)
fig.subplots_adjust(top = 0.97)
plt.suptitle('Linearity between each of Predictors and Flood Probability', fontsize = 14)
plt.show()

**Observations** - There is a **strong linear relationship** between each predictor to the target. The `FloodProbability` tends to increase as the predictor increases in their values. Hence, Linear Regression might be a good candidate for the prediction.

# Feature Engineering
**Credit to**
- https://www.kaggle.com/code/trupologhelper/ps4e5-openfe-blending-explain

Each newly created feature represents a combination or interaction between the original features, which can be more informative for predicting the likelihood of floods. Here's an explanation of each new feature:

- 'total': The sum of all original features for each data row. 📈
- 'mean': The average value of the original features for each data row. 🌡️
- 'std': The standard deviation of the original features for each data row. 📏
- 'max': The maximum value among the original features for each data row. 📈
- 'min': The minimum value among the original features for each data row. 📉
- 'median': The median of the original features for each data row. 📊
- 'ptp': The range (difference between the maximum and minimum values) of the original features for each data row. 📏
- 'q25': The 25th percentile (first quartile) of the original features for each data row. 📊
- 'q75': The 75th percentile (third quartile) of the original features for each data row. 📊
- 'ClimateImpact': The sum of monsoon intensity and climate change, indicating the overall impact of climatic factors. 🌍
- 'AnthropogenicPressure': The sum of deforestation, urbanization, agricultural practices, and encroachments, representing anthropogenic pressure on the environment. 🏭
- 'InfrastructureQuality': The sum of dam quality, drainage systems, and deteriorating infrastructure, indicating the overall quality of infrastructure. 🏗️
- 'CoastalVulnerabilityTotal': The sum of coastal vulnerability and landslides, representing the total vulnerability of coastal areas. 🌊
- 'PreventiveMeasuresEfficiency': The sum of river management, ineffective disaster preparedness, and inadequate planning, indicating the effectiveness of preventive measures. 🚧
- 'EcosystemImpact': The sum of wetland loss and watersheds, representing the impact on ecosystems. 🌿
- 'SocioPoliticalContext': The product of population assessment and political factors, indicating the socio-political context. 👥
- 'FloodVulnerabilityIndex': The average sum of anthropogenic pressure, infrastructure quality, total coastal vulnerability, and preventive measures efficiency, representing the flood vulnerability index. 🌊
- 'PopulationDensityImpact': The product of population assessment, urbanization, and encroachments, indicating the impact of population density. 👨‍👩‍👧‍👦
- 'DeforestationUrbanizationRatio': The ratio of deforestation to urbanization. 🌳🏙️
- 'AgriculturalEncroachmentImpact': The product of agricultural practices and encroachments, representing the impact of agricultural encroachments. 🚜
- 'DamDrainageInteraction': The product of dam quality and drainage systems, indicating the interaction between dams and drainage. 🏰🚰
- 'LandslideSiltationInteraction': The product of landslides and siltation, representing the interaction between landslides and siltation. ⛰️💧
- 'WatershedWetlandRatio': The ratio of watersheds to wetland loss. 🌊🌿
- 'PoliticalPreparednessInteraction': The product of political factors and ineffective disaster preparedness, indicating the interaction between politics and preparedness. 🏛️🚧
- 'TopographyDrainageSiltation': The sum of topographic drainage and siltation. 🗺️💧
- 'ClimateAnthropogenicInteraction': The product of climate impact and anthropogenic pressure, representing the interaction between climate and anthropogenic factors. 🌍🏭
- 'InfrastructurePreventionInteraction': The product of infrastructure quality and preventive measures efficiency, indicating the interaction between infrastructure and prevention. 🏗️🚧
- 'CoastalEcosystemInteraction': The product of total coastal vulnerability and ecosystem impact, representing the interaction between coastal areas and ecosystems. 🌊🌿

These new features are designed to capture various aspects and interactions that may influence the likelihood of floods, potentially improving the performance of the regression mode

In [ ]:
df_test.columns

In [ ]:
BASE_FEATURES = df_test.columns

def add_features(df):
    df['total'] = df[BASE_FEATURES].sum(axis=1)
    df['mean'] = df[BASE_FEATURES].mean(axis=1)
    df['std'] = df[BASE_FEATURES].std(axis=1)
    df['max'] = df[BASE_FEATURES].max(axis=1)
    df['min'] = df[BASE_FEATURES].min(axis=1)
    df['median'] = df[BASE_FEATURES].median(axis=1)
    df['ptp'] = df[BASE_FEATURES].values.ptp(axis=1)
    df['q25'] = df[BASE_FEATURES].quantile(0.25, axis=1)
    df['q75'] = df[BASE_FEATURES].quantile(0.75, axis=1)
    
    df['ClimateImpact'] = df['MonsoonIntensity'] + df['ClimateChange']
    df['AnthropogenicPressure'] = df['Deforestation'] + df['Urbanization'] + df['AgriculturalPractices'] + df['Encroachments']
    df['InfrastructureQuality'] = df['DamsQuality'] + df['DrainageSystems'] + df['DeterioratingInfrastructure']
    df['CoastalVulnerabilityTotal'] = df['CoastalVulnerability'] + df['Landslides']
    df['PreventiveMeasuresEfficiency'] = df['RiverManagement'] + df['IneffectiveDisasterPreparedness'] + df['InadequatePlanning']
    df['EcosystemImpact'] = df['WetlandLoss'] + df['Watersheds']
    df['SocioPoliticalContext'] = df['PopulationScore'] * df['PoliticalFactors']


    df['FloodVulnerabilityIndex'] = (df['AnthropogenicPressure'] + df['InfrastructureQuality'] +
                                     df['CoastalVulnerabilityTotal'] + df['PreventiveMeasuresEfficiency']) / 4
    
    df['PopulationDensityImpact'] = df['PopulationScore'] * (df['Urbanization'] + df['Encroachments'])
    
    df['DeforestationUrbanizationRatio'] = df['Deforestation'] / df['Urbanization']
    
    df['AgriculturalEncroachmentImpact'] = df['AgriculturalPractices'] * df['Encroachments']
    
    df['DamDrainageInteraction'] = df['DamsQuality'] * df['DrainageSystems']
    
    df['LandslideSiltationInteraction'] = df['Landslides'] * df['Siltation']
    
    df['WatershedWetlandRatio'] = df['Watersheds'] / df['WetlandLoss']
    
    df['PoliticalPreparednessInteraction'] = df['PoliticalFactors'] * df['IneffectiveDisasterPreparedness']
    
    
    df['TopographyDrainageSiltation'] = df['TopographyDrainage'] + df['Siltation']
    
    df['ClimateAnthropogenicInteraction'] = df['ClimateImpact'] * df['AnthropogenicPressure']
    
    df['InfrastructurePreventionInteraction'] = df['InfrastructureQuality'] * df['PreventiveMeasuresEfficiency']
    
    df['CoastalEcosystemInteraction'] = df['CoastalVulnerabilityTotal'] * df['EcosystemImpact']

    return df

df_train = add_features(df_train)
df_test = add_features(df_test)

In [ ]:
df_train.head()

# Machine Learning
### Preparation

In [ ]:
target = 'FloodProbability'
features = [col for col in df_train.columns if col != target ]

X = df_train[features]
y = df_train[target]

In [ ]:
# For machine learning
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import HistGradientBoostingRegressor,RandomForestRegressor,GradientBoostingRegressor,VotingRegressor
from sklearn.model_selection import KFold
from mlxtend.regressor import StackingCVRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR

### Preprocessing

In [ ]:
for column in X.columns:
    X[column].replace([np.inf, -np.inf], np.nan, inplace = True)
    mean = X[column].mean(skipna=True)
    X[column].fillna(mean, inplace = True)
X.isnull().any().sum()

### Model Selection
# Multiple Linear Regression

In [ ]:
lm = LinearRegression()

# Fit the data(train the model) 
lm.fit(X, y) 

# Predict
y_predicted = lm.predict(X)

# Model evaluation
r2 = r2_score(y, y_predicted) 

# printing values 
print('Slope:' ,lm.coef_) 
print('Intercept:', lm.intercept_) 
print('R2 score: ', r2) 

# XGBoost

In [ ]:
# Create XGBoost model
xgb = XGBRegressor(booster = 'gbtree',
                   max_depth = 10,
                   num_leaves = 250,
                   reg_alpha = 0.1,
                   reg_lambda = 3.25,
                   learning_rate = 0.01,
                   n_estimators = 3000,
                   subsample_for_bin= 165700, 
                   min_child_samples= 114, 
                   colsample_bytree= 0.9634,
                   subsample= 0.9592, 
                   random_state = 0)

n_splits = 5
# Create a KFold cross-validator
kf = KFold(n_splits = n_splits, shuffle = True, random_state = 42)

scores = []
# Perform K-Fold Cross-Validation
for train_index, val_index in kf.split(X):
    X_train, X_valid = X.iloc[train_index], X.iloc[val_index]
    y_train, y_valid = y[train_index], y[val_index]
    xgb.fit(X_train, y_train)
    y_pred = xgb.predict(X_valid)
    score = r2_score(y_valid, y_pred)
    print(score)
    scores.append(score)

# Output the average R2 score across all folds
print(f'Mean R2 score: {np.mean(scores):.5f}')

# CatBoost Regressor

In [ ]:
# Create CatBoostRegressor model
catb = CatBoostRegressor(n_estimators = 3000,
                       learning_rate = 0.05,
                       verbose = 0)

n_splits = 5
# Create a KFold cross-validator
kf = KFold(n_splits = n_splits, shuffle = True, random_state = 42)

scores = []
# Perform K-Fold Cross-Validation
for train_index, val_index in kf.split(X):
    X_train, X_valid = X.iloc[train_index], X.iloc[val_index]
    y_train, y_valid = y[train_index], y[val_index]
    catb.fit(X_train, y_train)
    y_pred = catb.predict(X_valid)
    score = r2_score(y_valid, y_pred)
    print(score)
    scores.append(score)

# Output the average R2 score across all folds
print(f'Mean R2 score: {np.mean(scores):.5f}')

# LightGBM

In [ ]:
'''n_splits = 5
# Create a KFold cross-validator
kf = KFold(n_splits = n_splits, shuffle = True, random_state = 42)
# Create XGBoost model
lgbm = LGBMRegressor(objective = 'regression',
               boosting_type = 'gbdt',
               max_depth = 10,
               num_leaves = 250,
               reg_alpha = 0.1,
               reg_lambda = 3.25,
               learning_rate = 0.01,
               n_estimators = 3000,
               subsample_for_bin= 165700, 
               min_child_samples= 114, 
               colsample_bytree= 0.9634,
               subsample= 0.9592, 
               random_state = 0,
               verbosity = -1)
scores = []
# Perform K-Fold Cross-Validation
for train_index, val_index in kf.split(X):
    X_train, X_valid = X.iloc[train_index], X.iloc[val_index]
    y_train, y_valid = y[train_index], y[val_index]
    lgbm.fit(X_train, y_train)
    y_pred = lgbm.predict(X_valid)
    score = r2_score(y_valid, y_pred)
    print(score)
    scores.append(score)

# Output the average R2 score across all folds
print(f'Mean R2 score: {np.mean(scores):.5f}')'''

# Stacking Ensemble
Use StackingCVRegressor to apply multiple regression methods to yield high accuracy and stability!

In [ ]:
'''r1 = catb
r2 = xgb
r3 = lgbm
r4 = lm
r5 = HistGradientBoostingRegressor(learning_rate = 0.05,
                                   max_iter = 400)
r6 = GradientBoostingRegressor(learning_rate = 0.05,
                               n_estimators = 400)
r7 = RandomForestRegressor(n_estimators = 400,
                           max_depth = 4)
r8 = SVR(kernel='linear')

stack = StackingCVRegressor(regressors=(r1, r2, r4, r5, r6, r7, r8),
                            meta_regressor = CatBoostRegressor(verbose = 0),
                            cv = KFold(n_splits=5))'''

In [ ]:
'''stack.fit(X, y)'''

In [ ]:
'''r1 = CatBoostRegressor(n_estimators = 1000,
                       learning_rate = 0.05,
                       verbose = 0)
r2 = xgb
r3 = lgb
r4 = HistGradientBoostingRegressor(learning_rate = 0.05,
                                   max_iter = 400)
r5 = GradientBoostingRegressor(learning_rate = 0.05,
                               n_estimators = 400)
r6 = RandomForestRegressor(n_estimators = 400,
                           max_depth = 4)
r7 = LinearRegression()
r8 = SVR(kernel='linear')

stack = StackingCVRegressor(regressors=(r1, r2, r3, r4,r5,r6,r7,r8),
                            meta_regressor = CatBoostRegressor(verbose = 0),
                            cv = KFold(n_splits=10))'''

In [ ]:
r1 = catb
r2 = xgb
#r3 = lgbm
r4 = HistGradientBoostingRegressor(learning_rate = 0.05,
                                   max_iter = 400)
r5 = GradientBoostingRegressor(learning_rate = 0.05,
                               n_estimators = 400)
r6 = RandomForestRegressor(n_estimators = 400,
                           max_depth = 4)
r7 = LinearRegression()
r8 = SVR(kernel='linear')

stack = StackingCVRegressor(regressors=(r1, r2, r4, r5, r6, r7, r8),
                            meta_regressor = CatBoostRegressor(verbose = 0),
                            cv = KFold(n_splits=10))

In [ ]:
'''n_splits = 5
# Create a KFold cross-validator
kf = KFold(n_splits = n_splits, shuffle = True, random_state = 42)

scores = []
# Perform K-Fold Cross-Validation
for train_index, val_index in kf.split(X):
    X_train, X_valid = X.iloc[train_index], X.iloc[val_index]
    y_train, y_valid = y[train_index], y[val_index]
    stack.fit(X_train, y_train)
    y_pred = stack.predict(X_valid)
    score = r2_score(y_valid, y_pred)
    print(score)
    scores.append(score)

# Output the average R2 score across all folds
print(f'Mean R2 score: {np.mean(scores):.5f}')'''

In [ ]:
stack.fit(X, y)

In [ ]:
'''for clf, label in zip([r1, r2, r3, r4,r5,r6,r7,r8, stack], ['CatBoostRegressor', 'XGBRegressor', 
                                                        'LGBMRegressor', 'HistGradientBoostingRegressor',
                                                         'GradientBoostingRegressor','RandomForestRegressor','LinearRegression',
                                                            'SVR','StackingCVRegressor']):
    clf.fit(X, y)
    scores = cross_val_score(clf, X, y, cv=2, scoring='r2')
    print("R2 Score: %0.2f (+/- %0.2f) [%s]" % (
        scores.mean(), scores.std(), label))'''

## Submission

In [ ]:
X_submission = df_test[features]

In [ ]:
'''y_xgb_pred = xgb.predict(X_submission)
#y_lgb_pred = lgbm.predict(X_submission)
y_cat_pred = catb.predict(X_submission)'''

In [ ]:
'''y_submission_pred = 0.5*y_xgb_pred + 0.5*y_cat_pred'''

In [ ]:
y_submission_pred = stack.predict(X_submission)

In [ ]:
df_test.reset_index(inplace = True)
df_test

In [ ]:
submission = pd.DataFrame({
    "id": df_test["id"],
    "probability": y_submission_pred,
}).set_index('id')

In [ ]:
submission.head()

In [ ]:
submission.to_csv("./submission.csv")